# ⛏️ OBD-II REAL DATA COLLECTION

## **Google Colab Notebook Overview**

This Colab notebook provides a comprehensive walkthrough of the **end-to-end processing pipeline** for OBD-II vehicle telemetry data, including:

1. **Raw Dataset Integration**  
   - Mounting Google Drive and loading raw OBD-II datasets from multiple sources, including KIT, Toyota Etios (GitHub), and the LEVIN Telematics dataset.  
   - Handling multiple CSV formats, file structures, and encoding variations across datasets.

2. **Data Cleaning & Preprocessing**  
   - Standardizing column names, resolving encoding anomalies (e.g., special characters like `Â`, `°C`), and converting timestamps.  
   - Forward-filling and interpolation of missing values where appropriate (based on <0.03% thresholds).  
   - Filtering out physically implausible values (e.g., negative RPM, extreme temperatures, or unrealistic speeds).

3. **Feature Engineering & Dataset Consolidation**  
   - Merging multiple CSVs by data source category (e.g., `drive`, `idle`, `live`) while preserving origin metadata.  
   - Conducting exploratory data analysis (EDA) to assess variable distributions, sensor reliability, and potential predictive features.  
   - Enriching the dataset with derived metrics (e.g., time deltas, trip-based aggregations, engine load ranges).

4. **Insight Extraction & Summary Reporting**  
   - Generating structured summary reports (in Markdown) detailing key statistical findings, sensor coverage, and missing data trends.  
   - Highlighting dataset suitability for time-series modeling, predictive maintenance, and anomaly detection.  
   - Providing recommendations for downstream machine learning or dashboard integration.

---

This notebook is intended to **guide the project team through the practical challenges and opportunities** of working with real-world vehicle OBD-II datasets. It sets the foundation for building robust, data-driven solutions for **fleet performance optimization and predictive maintenance.**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Download `LEVIN Vehicle Telematics Data – Yuñ Solutions` to Google Drive**

In [ ]:
!pip install -U gdown

import gdown
import shutil
import os

# Step 1: Download the ZIP from shared URL (Google Drive file ID)
file_id = "1T0M_7585gl8h-wcCyfBdDQVaTXKbVdw8"
output_path = "/content/levin_data.zip"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

# Step 2: Unzip the downloaded file
import zipfile
with zipfile.ZipFile(output_path, 'r') as zip_ref:
    zip_ref.extractall("/content/levin_data")

# Step 3: Move to target directory in Google Drive
target_dir = "/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data"
shutil.move("/content/levin_data", target_dir)

print(f"LEVIN dataset moved to {target_dir}")


Downloading...
From (original): https://drive.google.com/uc?id=1T0M_7585gl8h-wcCyfBdDQVaTXKbVdw8
From (redirected): https://drive.google.com/uc?id=1T0M_7585gl8h-wcCyfBdDQVaTXKbVdw8&confirm=t&uuid=08d0f977-ca29-4fab-acab-616269208395
To: /content/levin_data.zip
100%|██████████| 299M/299M [00:06<00:00, 44.4MB/s]


LEVIN dataset moved to /content/drive/My Drive/EAT40005/OBD-II/levin_data


# 📊 Data Exploration

## 1. **Automotive OBD-II Dataset – Karlsruhe Institute of Technology (KIT)**
- **Description:** This dataset contains CSV files with ten vehicle signals logged via the OBD-II interface. The data was recorded using the KIWI 3 OBD-II dongle from PLX Devices in combination with the OBD Auto Doctor app on iOS devices. The dataset includes parameters such as engine coolant temperature, intake manifold absolute pressure, engine RPM, vehicle speed, intake air temperature, mass air flow rate, absolute throttle position, ambient air temperature, and accelerator pedal positions D and E. Each file is named according to the date, vehicle brand and model, start and end positions, road conditions, and any special situations.​

- **Ideal For:** Analyzing real-world driving conditions and understanding vehicle behavior under various road scenarios.​

- **Access:** [Automotive OBD-II Dataset - KIT​](https://radar.kit.edu/radar/en/dataset/bCtGxdTklQlfQcAq)



### ***A. Exploration of Raw Files***

In [ ]:
import os
import pandas as pd

# Path to your KIT dataset
kit_path = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset'

# List and load all CSVs
kit_files = [os.path.join(kit_path, f) for f in os.listdir(kit_path) if f.endswith('.csv')]
print(f"Found {len(kit_files)} CSV files.")

# Preview each file deeply
for file in kit_files:
    print(f"\n📄 File: {os.path.basename(file)}")
    df = pd.read_csv(file)
    print("✅ Loaded successfully.")
    print("🔢 Shape:", df.shape)
    print("📌 Columns:", df.columns.tolist())
    print("📊 Data Types:\n", df.dtypes)
    print("❗ Missing Values:\n", df.isnull().sum())
    print("📈 Basic Statistics:\n", df.describe(include='all').transpose())
    print("🔍 Sample Rows:\n", df.sample(min(3, len(df))))

Streaming output truncated to the last 5000 lines.
       Intake Manifold Absolute Pressure [kPa]  Engine RPM [RPM]  \
9828                                      74.0            1288.0   
23470                                    102.0             899.0   
14693                                    103.0             979.0   

       Vehicle Speed Sensor [km/h]  Intake Air Temperature [Â°C]  \
9828                          22.0                          26.0   
23470                         15.0                          31.0   
14693                         17.0                          21.0   

       Air Flow Rate from Mass Flow Sensor [g/s]  \
9828                                       11.55   
23470                                       8.11   
14693                                      16.33   

       Absolute Throttle Position [%]  Ambient Air Temperature [Â°C]  \
9828                             25.1                           14.0   
23470                            83.5             

### ***B. Merging Identical CSV***

=> This found we have 2 identical files to be merged.

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# Path to KIT dataset
kit_path = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset'

# List all CSV files
kit_files = [f for f in os.listdir(kit_path) if f.endswith('.csv')]
print(f"📁 Found {len(kit_files)} CSV files.")

# Step 1: Group files by identical column sets
schema_groups = defaultdict(list)

for file in kit_files:
    file_path = os.path.join(kit_path, file)
    try:
        df = pd.read_csv(file_path, nrows=1)  # Read just the first row for columns
        col_signature = tuple(df.columns)
        schema_groups[col_signature].append(file)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")

print(f"✅ Found {len(schema_groups)} unique schema groups.")

# Step 2: Merge files with identical schema
output_dir = kit_path  # Output back to same folder
for i, (schema, files) in enumerate(schema_groups.items(), start=1):
    if len(files) > 1:
        print(f"\n🔄 Merging group {i} with files: {files}")
        combined = []
        for file in files:
            file_path = os.path.join(kit_path, file)
            df = pd.read_csv(file_path)
            df['source_file'] = file
            combined.append(df)
        merged_df = pd.concat(combined, ignore_index=True)
        output_path = os.path.join(output_dir, f"combined_{i}.csv")
        merged_df.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}, shape: {merged_df.shape}")

📁 Found 81 CSV files.
✅ Found 2 unique schema groups.

🔄 Merging group 1 with files: ['2017-07-11_Seat_Leon_S_RT_Frei.csv', '2017-08-05_Seat_Leon_S_CW_Normal.csv', '2017-08-07_Seat_Leon_KA_RT_Frei.csv', '2017-08-10_Seat_Leon_S_RT_Frei.csv', '2017-07-26_Seat_Leon_S_KA_Normal.csv', '2017-07-13_Seat_Leon_KA_KA_Normal.csv', '2017-08-08_Seat_Leon_RT_S_Normal.csv', '2017-07-14_Seat_Leon_KA_KA_Frei.csv', '2017-07-27_Seat_Leon_KA_KA_Normal.csv', '2017-07-24_Seat_Leon_RT_KA_Normal.csv', '2017-07-31_Seat_Leon_KA_KA_Frei.csv', '2017-07-28_Seat_Leon_KA_RT_Stau.csv', '2017-08-07_Seat_Leon_RT_KA_Frei_Vollbremsung.csv', '2017-07-28_Seat_Leon_KA_KA_Normal.csv', '2017-08-01_Seat_Leon_KA_KA_Frei.csv', '2017-07-05_Seat_Leon_RT_S_Stau.csv', '2017-07-10_Seat_Leon_KA_KA_Stau.csv', '2017-08-10_Seat_Leon_RT_S_Normal.csv', '2017-08-01_Seat_Leon_KA_RT_Normal.csv', '2017-08-02_Seat_Leon_RT_S_Normal.csv', '2017-07-05_Seat_Leon_S_KA_Normal.csv', '2017-07-06_Seat_Leon_KA_RT_Normal.csv', '2017-08-09_Seat_Leon_S_RT_N

###***C. Explore Merging File***

In [ ]:
import os
import pandas as pd

# Path to your KIT dataset with combined files
kit_path = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset'

# List only combined_*.csv files
combined_files = [os.path.join(kit_path, f) for f in os.listdir(kit_path) if f.startswith('combined_') and f.endswith('.csv')]
print(f"🗂️ Found {len(combined_files)} combined CSV files.")

# Explore each combined file
for file in combined_files:
    print(f"\n📄 Exploring File: {os.path.basename(file)}")
    try:
        df = pd.read_csv(file)
        print("✅ Loaded successfully.")
        print("🔢 Shape:", df.shape)
        print("📌 Columns:", df.columns.tolist())
        print("📊 Data Types:\n", df.dtypes)
        print("❗ Missing Values:\n", df.isnull().sum())
        print("📈 Basic Statistics:\n", df.describe(include='all').transpose())
        print("🔍 Sample Rows:\n", df.sample(min(3, len(df))))
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")


🗂️ Found 2 combined CSV files.

📄 Exploring File: combined_1.csv
✅ Loaded successfully.
🔢 Shape: (1262424, 12)
📌 Columns: ['Time', 'Engine Coolant Temperature [Â°C]', 'Intake Manifold Absolute Pressure [kPa]', 'Engine RPM [RPM]', 'Vehicle Speed Sensor [km/h]', 'Intake Air Temperature [Â°C]', 'Air Flow Rate from Mass Flow Sensor [g/s]', 'Absolute Throttle Position [%]', 'Ambient Air Temperature [Â°C]', 'Accelerator Pedal Position D [%]', 'Accelerator Pedal Position E [%]', 'source_file']
📊 Data Types:
 Time                                          object
Engine Coolant Temperature [Â°C]               int64
Intake Manifold Absolute Pressure [kPa]      float64
Engine RPM [RPM]                             float64
Vehicle Speed Sensor [km/h]                  float64
Intake Air Temperature [Â°C]                 float64
Air Flow Rate from Mass Flow Sensor [g/s]    float64
Absolute Throttle Position [%]               float64
Ambient Air Temperature [Â°C]                float64
Accelerator Peda

### **D. Summary of Merged Files**

#### 📊 **OBD-II KIT Dataset Summary Report (Merged Files)**

**Dataset Location**:  
`/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset/`

---

##### ✅ `combined_1.csv`

- **Shape:** 1,262,424 rows × 12 columns
- **Columns:**
  - Time
  - Engine Coolant Temperature [°C]
  - Intake Manifold Absolute Pressure [kPa]
  - Engine RPM [RPM]
  - Vehicle Speed Sensor [km/h]
  - Intake Air Temperature [°C]
  - Air Flow Rate from Mass Flow Sensor [g/s]
  - Absolute Throttle Position [%]
  - Ambient Air Temperature [°C]
  - Accelerator Pedal Position D [%]
  - Accelerator Pedal Position E [%]
  - `source_file` (added for origin tracking)

- **Missing Values:** Present in 10/12 columns; most have < 0.03% missing values
  - Example: `Accelerator Pedal Position E [%]` has 351 missing entries

- **Key Observations:**
  - Coolant Temp (mean): **84.95°C**
  - RPM (mean): **1481.64**, range: 0 – 3788
  - Speed (mean): **62.49 km/h**, range: 0 – 218
  - Air Temp (mean): **30.33°C**
  - Throttle Position (mean): **80.60%**
  - Ambient Air Temp: Mean **20.57°C**, Range **12–33°C**

- **Notable Sample Entry:**
  ```plaintext
  Time: 07:51:39.317 | RPM: 2617 | Speed: 155 km/h | Throttle: 83.5% | Source: 2017-07-13_Seat_Leon_RT_KA_Stau_Messfehler.csv
  ```

---

##### ✅ `combined_2.csv`

- **Shape:** 1,431,400 rows × 12 columns
- **Columns:** Identical to `combined_1.csv`, with consistent schema

- **Missing Values:** Also minimal
  - Example: `Accelerator Pedal Position E [%]` has 374 missing values

- **Key Observations:**
  - Coolant Temp (mean): **78.53°C**, slightly cooler than `combined_1`
  - RPM (mean): **1506.10**, slightly higher than `combined_1`
  - Speed (mean): **62.04 km/h**
  - Intake Air Temp (mean): **9.90°C** → significantly lower, likely colder ambient conditions
  - Ambient Air Temp: Mean **1.09°C**, range **-14 to 21°C** → much wider and colder than `combined_1`
  - Throttle Position: Mean **81.81%**

- **Notable Sample Entry:**
  ```plaintext
  Time: 18:52:52.689 | RPM: 1381 | Speed: 54 km/h | Throttle: 83.5% | Ambient Temp: 9°C | Source: 2018-03-29_Seat_Leon_KA_RT_Stau.csv
  ```

---

##### 📌 Key Takeaways

- Both combined datasets contain rich vehicle telemetry (RPM, throttle, temperature, speed).
- `combined_2.csv` represents colder climate conditions, suggesting data recorded in winter or cold regions.
- Minimal missing data — excellent for time-series and ML preprocessing.
- `source_file` column enables easy segmentation by trip, date, or driving condition.

### ***E. Data Cleaning***

In [ ]:
import pandas as pd
import numpy as np

# Load dataset (change to combined_1.csv or combined_2.csv as needed)
file_path = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset/combined_2.csv'
df = pd.read_csv(file_path)

# 1. Clean and standardize column names
df.columns = df.columns.str.strip() \
    .str.replace('Â', '') \
    .str.replace('°', '°C') \
    .str.replace('[\[\]]', '', regex=True) \
    .str.replace('°CC', '°C')  # Fix double encoding issue

# 2. Convert 'Time' column to datetime (if not already)
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S.%f', errors='coerce')

# 3. Handle missing values (<0.03% across all relevant columns)
print("Missing Data (%):\n", df.isnull().mean() * 100)
df.ffill(inplace=True)  # Forward fill

# 4. Remove outliers or physically invalid values
df = df[
    (df['Engine Coolant Temperature °C'] >= -40) & (df['Engine Coolant Temperature °C'] <= 130) &
    (df['Vehicle Speed Sensor km/h'] >= 0) & (df['Vehicle Speed Sensor km/h'] <= 250) &
    (df['Engine RPM RPM'] >= 0) & (df['Engine RPM RPM'] <= 8000)
]

# 5. Reset index after cleaning
df.reset_index(drop=True, inplace=True)

# 6. Save cleaned dataset
cleaned_path = file_path.replace('.csv', '_cleaned.csv')
df.to_csv(cleaned_path, index=False)
print(f"✅ Cleaned file saved to: {cleaned_path}")

Missing Data (%):
 Time                                       0.000000
Engine Coolant Temperature °C              0.000489
Intake Manifold Absolute Pressure kPa      0.003423
Engine RPM RPM                             0.006357
Vehicle Speed Sensor km/h                  0.008593
Intake Air Temperature °C                  0.011527
Air Flow Rate from Mass Flow Sensor g/s    0.014461
Absolute Throttle Position %               0.017396
Ambient Air Temperature °C                 0.020330
Accelerator Pedal Position D %             0.023893
Accelerator Pedal Position E %             0.026128
source_file                                0.000000
dtype: float64
✅ Cleaned file saved to: /content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset/combined_2_cleaned.csv


In [ ]:
# Remeain Feature after cleaning
file_1 = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset/combined_1_cleaned.csv'
file_2 = '/content/drive/My Drive/EAT40005/OBD-II/Karlsruhe Institute of Technology (KIT)/data/dataset/OBD-II-Dataset/combined_2_cleaned.csv'
df1 = pd.read_csv(file_1)
df2 = pd.read_csv(file_1)
print("-- Cleaned Dataset 1 --")
print(df1.columns.tolist())
print("-- Cleaned Dataset 2 --")
print(df2.columns.tolist())

-- Cleaned Dataset 1 --
['Time', 'Engine Coolant Temperature °C', 'Intake Manifold Absolute Pressure kPa', 'Engine RPM RPM', 'Vehicle Speed Sensor km/h', 'Intake Air Temperature °C', 'Air Flow Rate from Mass Flow Sensor g/s', 'Absolute Throttle Position %', 'Ambient Air Temperature °C', 'Accelerator Pedal Position D %', 'Accelerator Pedal Position E %', 'source_file']
-- Cleaned Dataset 2 --
['Time', 'Engine Coolant Temperature °C', 'Intake Manifold Absolute Pressure kPa', 'Engine RPM RPM', 'Vehicle Speed Sensor km/h', 'Intake Air Temperature °C', 'Air Flow Rate from Mass Flow Sensor g/s', 'Absolute Throttle Position %', 'Ambient Air Temperature °C', 'Accelerator Pedal Position D %', 'Accelerator Pedal Position E %', 'source_file']


## 2. **OBD-II Dataset – GitHub (Toyota Etios 2014)**

- **Description:** This dataset was collected during a master's thesis project and comprises 27 different vehicle parameters (PIDs) from a 2014 Toyota Etios. Data acquisition was performed using an embedded system connected to the vehicle's OBD-II interface. Parameters include engine run time, RPM, vehicle speed, throttle position, engine load, coolant temperature, fuel trims, intake manifold pressure, fuel tank level, accelerator pedal positions, commanded throttle actuator, air-fuel ratio, barometric pressure, and more. The dataset also distinguishes between driving modes, such as motion and idle.​
radar-service.eu

- **Ideal For:** Detailed analysis of engine performance and driver behavior in a specific vehicle model.​

- **Access:** [carOBD Dataset – GitHub​](https://github.com/eron93br/carOBD)



### ***A. Exploration of Raw Data***

In [ ]:
import os
import pandas as pd

toyota_path = '/content/drive/My Drive/EAT40005/OBD-II/Toyota Etios 2014'
file_groups = {'drive': [], 'idle': [], 'live': [], 'long': [], 'ufpe': []}

# Group files by prefix
for file in os.listdir(toyota_path):
    if file.endswith('.csv'):
        for key in file_groups:
            if file.startswith(key):
                file_groups[key].append(os.path.join(toyota_path, file))
                break

# Explore each group
for category, files in file_groups.items():
    print(f"\n🔍 Exploring '{category}' group with {len(files)} files.")
    sample_file = files[0]
    df = pd.read_csv(sample_file)
    print(f"Sample file: {os.path.basename(sample_file)}")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("Data types:\n", df.dtypes)
    print("Missing values:\n", df.isnull().sum())
    print("Head rows:\n", df.head())


🔍 Exploring 'drive' group with 13 files.
Sample file: drive5.csv
Shape: (627, 27)
Columns: ['ENGINE_RUN_TINE ()', 'ENGINE_RPM ()', 'VEHICLE_SPEED ()', 'THROTTLE ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'LONG_TERM_FUEL_TRIM_BANK_1 ()', 'SHORT_TERM_FUEL_TRIM_BANK_1 ()', 'INTAKE_MANIFOLD_PRESSURE ()', 'FUEL_TANK ()', 'ABSOLUTE_THROTTLE_B ()', 'PEDAL_D ()', 'PEDAL_E ()', 'COMMANDED_THROTTLE_ACTUATOR ()', 'FUEL_AIR_COMMANDED_EQUIV_RATIO ()', 'ABSOLUTE_BAROMETRIC_PRESSURE ()', 'RELATIVE_THROTTLE_POSITION ()', 'INTAKE_AIR_TEMP ()', 'TIMING_ADVANCE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()', 'CONTROL_MODULE_VOLTAGE ()', 'COMMANDED_EVAPORATIVE_PURGE ()', 'TIME_RUN_WITH_MIL_ON ()', 'TIME_SINCE_TROUBLE_CODES_CLEARED ()', 'DISTANCE_TRAVELED_WITH_MIL_ON ()', 'WARM_UPS_SINCE_CODES_CLEARED ()']
Data types:
 ENGINE_RUN_TINE ()                       float64
ENGINE_RPM ()                            float64
VEHICLE_SPEED ()                         float

### ***B. Merging Typed Set***

In [ ]:
# Function to merge all files in a group
def merge_group(file_list, group_name, save_path):
    dfs = []
    for file in file_list:
        df = pd.read_csv(file)
        df['source_type'] = group_name  # Add source type
        dfs.append(df)
    merged_df = pd.concat(dfs, ignore_index=True)
    merged_df.to_csv(os.path.join(save_path, f'{group_name}_merged.csv'), index=False)
    print(f"✅ Merged {group_name} group into {group_name}_merged.csv, shape: {merged_df.shape}")

save_path = toyota_path  # Same folder for now
for group, files in file_groups.items():
    merge_group(files, group, save_path)

✅ Merged drive group into drive_merged.csv, shape: (39779, 28)
✅ Merged idle group into idle_merged.csv, shape: (72226, 28)
✅ Merged live group into live_merged.csv, shape: (97281, 28)
✅ Merged long group into long_merged.csv, shape: (60947, 28)
✅ Merged ufpe group into ufpe_merged.csv, shape: (34066, 28)


### ***C. Exploration on Merged Sets***

In [ ]:
import os
import pandas as pd

# Path to your KIT dataset with combined files
toyota_path = '/content/drive/My Drive/EAT40005/OBD-II/Toyota Etios 2014'

# List only combined_*.csv files
combined_files = [os.path.join(toyota_path, f)
                  for f in os.listdir(toyota_path)
                  if f.endswith('_merged.csv')]
print(f"🗂️ Found {len(combined_files)} combined CSV files.")

# Explore each combined file
for file in combined_files:
    print(f"\n📄 Exploring File: {os.path.basename(file)}")
    try:
        df = pd.read_csv(file)
        print("✅ Loaded successfully.")
        print("🔢 Shape:", df.shape)
        print("📌 Columns:", df.columns.tolist())
        print("📊 Data Types:\n", df.dtypes)
        print("❗ Missing Values:\n", df.isnull().sum())
        print("📈 Basic Statistics:\n", df.describe(include='all').transpose())
        print("🔍 Sample Rows:\n", df.sample(min(3, len(df))))
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")


🗂️ Found 5 combined CSV files.

📄 Exploring File: drive_merged.csv
✅ Loaded successfully.
🔢 Shape: (39779, 28)
📌 Columns: ['ENGINE_RUN_TINE ()', 'ENGINE_RPM ()', 'VEHICLE_SPEED ()', 'THROTTLE ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'LONG_TERM_FUEL_TRIM_BANK_1 ()', 'SHORT_TERM_FUEL_TRIM_BANK_1 ()', 'INTAKE_MANIFOLD_PRESSURE ()', 'FUEL_TANK ()', 'ABSOLUTE_THROTTLE_B ()', 'PEDAL_D ()', 'PEDAL_E ()', 'COMMANDED_THROTTLE_ACTUATOR ()', 'FUEL_AIR_COMMANDED_EQUIV_RATIO ()', 'ABSOLUTE_BAROMETRIC_PRESSURE ()', 'RELATIVE_THROTTLE_POSITION ()', 'INTAKE_AIR_TEMP ()', 'TIMING_ADVANCE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()', 'CONTROL_MODULE_VOLTAGE ()', 'COMMANDED_EVAPORATIVE_PURGE ()', 'TIME_RUN_WITH_MIL_ON ()', 'TIME_SINCE_TROUBLE_CODES_CLEARED ()', 'DISTANCE_TRAVELED_WITH_MIL_ON ()', 'WARM_UPS_SINCE_CODES_CLEARED ()', 'source_type']
📊 Data Types:
 ENGINE_RUN_TINE ()                       float64
ENGINE_RPM ()                            float64

<ipython-input-10-d3b9f893a533>:17: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


✅ Loaded successfully.
🔢 Shape: (72226, 28)
📌 Columns: ['ENGINE_RUN_TINE ()', 'ENGINE_RPM ()', 'VEHICLE_SPEED ()', 'THROTTLE ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'LONG_TERM_FUEL_TRIM_BANK_1 ()', 'SHORT_TERM_FUEL_TRIM_BANK_1 ()', 'INTAKE_MANIFOLD_PRESSURE ()', 'FUEL_TANK ()', 'ABSOLUTE_THROTTLE_B ()', 'PEDAL_D ()', 'PEDAL_E ()', 'COMMANDED_THROTTLE_ACTUATOR ()', 'FUEL_AIR_COMMANDED_EQUIV_RATIO ()', 'ABSOLUTE_BAROMETRIC_PRESSURE ()', 'RELATIVE_THROTTLE_POSITION ()', 'INTAKE_AIR_TEMP ()', 'TIMING_ADVANCE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()', 'CONTROL_MODULE_VOLTAGE ()', 'COMMANDED_EVAPORATIVE_PURGE ()', 'TIME_RUN_WITH_MIL_ON ()', 'TIME_SINCE_TROUBLE_CODES_CLEARED ()', 'DISTANCE_TRAVELED_WITH_MIL_ON ()', 'WARM_UPS_SINCE_CODES_CLEARED ()', 'source_type']
📊 Data Types:
 ENGINE_RUN_TINE ()                       float64
ENGINE_RPM ()                            float64
VEHICLE_SPEED ()                         float64
THROTTLE ()      

<ipython-input-10-d3b9f893a533>:17: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


✅ Loaded successfully.
🔢 Shape: (97281, 28)
📌 Columns: ['ENGINE_RUN_TINE ()', 'ENGINE_RPM ()', 'VEHICLE_SPEED ()', 'THROTTLE ()', 'ENGINE_LOAD ()', 'COOLANT_TEMPERATURE ()', 'LONG_TERM_FUEL_TRIM_BANK_1 ()', 'SHORT_TERM_FUEL_TRIM_BANK_1 ()', 'INTAKE_MANIFOLD_PRESSURE ()', 'FUEL_TANK ()', 'ABSOLUTE_THROTTLE_B ()', 'PEDAL_D ()', 'PEDAL_E ()', 'COMMANDED_THROTTLE_ACTUATOR ()', 'FUEL_AIR_COMMANDED_EQUIV_RATIO ()', 'ABSOLUTE_BAROMETRIC_PRESSURE ()', 'RELATIVE_THROTTLE_POSITION ()', 'INTAKE_AIR_TEMP ()', 'TIMING_ADVANCE ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR1 ()', 'CATALYST_TEMPERATURE_BANK1_SENSOR2 ()', 'CONTROL_MODULE_VOLTAGE ()', 'COMMANDED_EVAPORATIVE_PURGE ()', 'TIME_RUN_WITH_MIL_ON ()', 'TIME_SINCE_TROUBLE_CODES_CLEARED ()', 'DISTANCE_TRAVELED_WITH_MIL_ON ()', 'WARM_UPS_SINCE_CODES_CLEARED ()', 'source_type']
📊 Data Types:
 ENGINE_RUN_TINE ()                       float64
ENGINE_RPM ()                            float64
VEHICLE_SPEED ()                         float64
THROTTLE ()      

<ipython-input-10-d3b9f893a533>:17: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


### ***D. Summary***

#### 📊 **Toyota Etios OBD-II Dataset Summary Report (Merged Files)**

**Dataset Location**:  
`/content/drive/My Drive/EAT40005/OBD-II/Toyota Etios 2014/`

---

##### ✅ `drive_merged.csv`

- **Shape:** 39,779 rows × 28 columns
- **Key Observations:**
  - **RPM:** Mean = 28.15 (very low), suggesting low-load conditions
  - **Coolant Temp:** Mean = -0.61°C, Min = -7.81°C, likely sensor calibration or cold starts
  - **Speed:** Avg = 19.77 km/h, typical of urban driving
  - **Throttle:** Avg = 35.59%, Range = 0–100%
  - **Commanded Throttle Actuator** is always 0.0
  - `WARM_UPS_SINCE_CODES_CLEARED ()`: Completely missing (NaN in all rows)
- **Missing Data:** Mostly clean except one fully null column
- **Source Type:** drive

---

##### ✅ `idle_merged.csv`

- **Shape:** 72,226 rows × 28 columns
- **Key Observations:**
  - **RPM:** Always 0.0, as expected from idle condition
  - **Speed:** Avg = 16.88 km/h, suggesting coasting or parking idle
  - **Throttle:** Avg = 29.36%
  - **Coolant Temp:** Avg = -0.50°C, anomalies persist
  - **Commanded Throttle Actuator:** 0.0 (all rows)
  - **ABSOLUTE_BAROMETRIC_PRESSURE ():** Object type, high freq of '0.0'
- **Missing Data:** Negligible (1 missing in many columns), but 1 full-null column
- **Source Type:** idle

---

##### ✅ `live_merged.csv`

- **Shape:** 97,281 rows × 28 columns
- **Key Observations:**
  - **RPM:** Mean = 228.39, Max = 2500, full range present
  - **Throttle:** Avg = 32.94%
  - **Coolant Temp:** Wide variation, Avg = 11.96°C
  - **Control Module Voltage:** Avg = 20.47V, Max = 100V
  - **RELATIVE_THROTTLE_POSITION ():** Object dtype with 173 unique values
  - **Significant missing in `WARM_UPS`, `DISTANCE_TRAVELED_WITH_MIL_ON ()`, `TIME_SINCE_TROUBLE_CODES_CLEARED ()`
- **Source Type:** live

---

##### ✅ `long_merged.csv`

- **Shape:** 60,947 rows × 28 columns
- **Key Observations:**
  - **Speed:** Avg = 20.81 km/h, likely longer city trips
  - **Coolant Temp:** Avg = -0.86°C, likely cold engine logging
  - **Throttle:** Avg = 38.96%, Range up to 100%
  - **Commanded Throttle Actuator:** Always 0.0
  - `WARM_UPS_SINCE_CODES_CLEARED ()`: Fully missing
- **Source Type:** long

---

##### ✅ `ufpe_merged.csv`

- **Shape:** 34,066 rows × 28 columns
- **Key Observations:**
  - **RPM:** Avg = 22.53, suggesting low-speed short trips
  - **Throttle:** Avg = 34.23%, reasonable range
  - **Coolant Temp:** Avg = -0.15°C
  - **Control Module Voltage:** 0.0 in ~60% of rows (object dtype)
  - `WARM_UPS_SINCE_CODES_CLEARED ()`: All missing
- **Source Type:** ufpe

---

##### 📌 Key Takeaways

- All files share **identical schema** with consistent OBD-II signal types.
- `WARM_UPS_SINCE_CODES_CLEARED ()` is **completely missing** across all categories.
- `Control Module Voltage` and `RELATIVE_THROTTLE_POSITION ()` have mixed or invalid types in multiple categories and may require cleaning.
- **Drive types (idle/live/drive/long/ufpe)** vary by trip conditions. Can be used as a target for unsupervised behavior clustering.
- **Throttle, RPM, Coolant Temp, and Load** offer rich predictive signals for maintenance.
- Dataset offers **strong coverage of real driving conditions** but will need:
  - Data cleaning for types and nulls
  - Scaling/normalization for model input
  - Possibly merging sensor signals temporally

## 3. **LEVIN Vehicle Telematics Data – Yuñ Solutions**
- **Description:** Collected over four months from approximately 30 vehicles, this dataset includes OBD-II data at 1Hz frequency and accelerometer data at 25Hz. It encompasses a wide range of parameters such as device ID, timestamp, trip ID, accelerometer data, GPS speed, battery voltage, coolant temperature, diagnostic trouble codes, engine load, intake air temperature, manifold absolute pressure, calculated mileage, mass airflow, engine RPM, OBD speed, timing advance, throttle position, and magnetometer data. The dataset is available in multiple formats, including CSV and SQLite3.​

- **Ideal For:** Comprehensive analysis combining OBD-II data with motion sensors for advanced vehicle diagnostics and driver behavior studies.​

- **Access:** [LEVIN Open Data – GitHub](https://github.com/YunSolutions/levin-openData)

### ***A. Exploration on Raw Data***

In [ ]:
import os
import pandas as pd

levin_path = '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data'
levin_files = sorted([os.path.join(levin_path, f) for f in os.listdir(levin_path) if f.endswith('.csv')])

print(f"📁 Total LEVIN CSVs: {len(levin_files)}")

# Deep inspection of several files (first, middle, last)
sampled_files = [levin_files[0], levin_files[len(levin_files)//2], levin_files[-1]]

for file in sampled_files:
    print(f"\n🔍 Inspecting file: {os.path.basename(file)}")
    df = pd.read_csv(file)
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print(df.dtypes)
    print("Missing values:\n", df.isnull().sum())
    print("Basic stats:\n", df.describe(include='all').transpose())
    print("Sample rows:\n", df.sample(min(3, len(df))))

📁 Total LEVIN CSVs: 29

🔍 Inspecting file: 1.csv
Shape: (117275, 20)
Columns: ['Unnamed: 0', 'timeStamp', 'tripID', 'gps_speed', 'battery', 'cTemp', 'dtc', 'eLoad', 'iat', 'imap', 'kpl', 'maf', 'rpm', 'speed', 'tAdv', 'tPos', 'accData', 'mx', 'my', 'mz']
Unnamed: 0      int64
timeStamp      object
tripID          int64
gps_speed     float64
battery       float64
cTemp         float64
dtc           float64
eLoad         float64
iat           float64
imap          float64
kpl           float64
maf           float64
rpm           float64
speed           int64
tAdv          float64
tPos          float64
accData        object
mx            float64
my            float64
mz            float64
dtype: object
Missing values:
 Unnamed: 0         0
timeStamp          0
tripID             0
gps_speed          0
battery            0
cTemp         117275
dtc           117275
eLoad              0
iat           117275
imap          117275
kpl                0
maf                0
rpm                0
s

### ***B. Merging***

In [ ]:
import os
import pandas as pd
from collections import defaultdict

# Path to KIT dataset
levin_path = '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data'
levin_files = sorted([os.path.join(levin_path, f) for f in os.listdir(levin_path) if f.endswith('.csv')])

# List all CSV files
print(f"📁 Found {len(levin_files)} CSV files.")

# Step 1: Group files by identical column sets
schema_groups = defaultdict(list)

for file in levin_files:
    file_path = os.path.join(levin_path, file)
    try:
        df = pd.read_csv(file_path, nrows=1)  # Read just the first row for columns
        col_signature = tuple(df.columns)
        schema_groups[col_signature].append(file)
    except Exception as e:
        print(f"❌ Failed to read {file}: {e}")

print(f"✅ Found {len(schema_groups)} unique schema groups.")

# Step 2: Merge files with identical schema
output_dir = levin_path  # Output back to same folder
for i, (schema, files) in enumerate(schema_groups.items(), start=1):
    if len(files) > 1:
        print(f"\n🔄 Merging group {i} with files: {files}")
        combined = []
        for file in files:
            file_path = os.path.join(levin_path, file)
            df = pd.read_csv(file_path)
            df['source_file'] = file
            combined.append(df)
        merged_df = pd.concat(combined, ignore_index=True)
        output_path = os.path.join(output_dir, f"combined_{i}.csv")
        merged_df.to_csv(output_path, index=False)
        print(f"✅ Saved: {output_path}, shape: {merged_df.shape}")

📁 Found 29 CSV files.
✅ Found 2 unique schema groups.

🔄 Merging group 1 with files: ['/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/1.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/11.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/12.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/13.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/19.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/2.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/23.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/34.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/35.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/36.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/37.csv', '/content/drive/My Drive/EAT40005/OB

<ipython-input-11-4260b68a4777>:34: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)
<ipython-input-11-4260b68a4777>:34: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


✅ Saved: /content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/combined_1.csv, shape: (5892307, 21)

🔄 Merging group 2 with files: ['/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/15.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/16.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/20.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/22.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/24.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/28.csv', '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/50.csv']
✅ Saved: /content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/combined_2.csv, shape: (1322388, 18)


###***C. Explore Merging File***

In [ ]:
import os
import pandas as pd

# Path to your KIT dataset with combined files
levin_path = '/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data'

# List only combined_*.csv files
combined_files = [os.path.join(levin_path, f) for f in os.listdir(levin_path) if f.startswith('combined_') and f.endswith('.csv')]
print(f"🗂️ Found {len(combined_files)} combined CSV files.")

# Explore each combined file
for file in combined_files:
    print(f"\n📄 Exploring File: {os.path.basename(file)}")
    try:
        df = pd.read_csv(file)
        print("✅ Loaded successfully.")
        print("🔢 Shape:", df.shape)
        print("📌 Columns:", df.columns.tolist())
        print("📊 Data Types:\n", df.dtypes)
        print("❗ Missing Values:\n", df.isnull().sum())
        print("📈 Basic Statistics:\n", df.describe(include='all').transpose())
        print("🔍 Sample Rows:\n", df.sample(min(3, len(df))))
    except Exception as e:
        print(f"❌ Error loading {file}: {e}")


🗂️ Found 2 combined CSV files.

📄 Exploring File: combined_1.csv


<ipython-input-12-13463f569792>:15: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


✅ Loaded successfully.
🔢 Shape: (5892307, 21)
📌 Columns: ['Unnamed: 0', 'timeStamp', 'tripID', 'gps_speed', 'battery', 'cTemp', 'dtc', 'eLoad', 'iat', 'imap', 'kpl', 'maf', 'rpm', 'speed', 'tAdv', 'tPos', 'accData', 'mx', 'my', 'mz', 'source_file']
📊 Data Types:
 Unnamed: 0       int64
timeStamp       object
tripID           int64
gps_speed      float64
battery        float64
cTemp          float64
dtc            float64
eLoad          float64
iat            float64
imap           float64
kpl            float64
maf            float64
rpm            float64
speed          float64
tAdv           float64
tPos           float64
accData         object
mx             float64
my             float64
mz             float64
source_file     object
dtype: object
❗ Missing Values:
 Unnamed: 0           0
timeStamp            0
tripID               0
gps_speed      1183038
battery        1183038
cTemp          4186387
dtc            4186387
eLoad          1183038
iat            4186387
imap         

<ipython-input-12-13463f569792>:15: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


✅ Loaded successfully.
🔢 Shape: (1322388, 18)
📌 Columns: ['Unnamed: 0', 'timeStamp', 'tripID', 'gps_speed', 'battery', 'cTemp', 'dtc', 'eLoad', 'iat', 'imap', 'kpl', 'maf', 'rpm', 'speed', 'tAdv', 'tPos', 'accData', 'source_file']
📊 Data Types:
 Unnamed: 0       int64
timeStamp       object
tripID           int64
gps_speed      float64
battery        float64
cTemp          float64
dtc            float64
eLoad          float64
iat            float64
imap           float64
kpl            float64
maf            float64
rpm            float64
speed          float64
tAdv           float64
tPos           float64
accData         object
source_file     object
dtype: object
❗ Missing Values:
 Unnamed: 0          0
timeStamp           0
tripID              0
gps_speed      929108
battery        929108
cTemp          929108
dtc            929108
eLoad          929108
iat            929108
imap           929108
kpl            929108
maf            929108
rpm            929108
speed               0

### **D. Summary of Merged Files**

#### 📊 **LEVIN Vehicle Telematics Dataset Summary Report (Merged Files)**

**Dataset Location:**  
`/content/drive/My Drive/EAT40005/OBD-II/LEVIN Vehicle Telematics Data/`

---

##### ✅ `combined_1.csv`

- **Shape:** 5,892,307 rows × 21 columns
- **Columns:**
  - timeStamp
  - tripID
  - gps_speed
  - battery
  - cTemp
  - dtc
  - eLoad
  - iat
  - imap
  - kpl
  - maf
  - rpm
  - speed
  - tAdv
  - tPos
  - accData
  - mx, my, mz (3-axis motion sensors)
  - `source_file` (origin tracking)
  - `Unnamed: 0` (auto-generated index)

- **Missing Values:**  
  Present in ~10/21 columns, notably:
  - `gps_speed`, `battery`, `eLoad`, `rpm`: ~1.18M missing (~20%)
  - `cTemp`, `dtc`, `iat`, `imap`, `tAdv`: ~4.2M missing (~70%)

- **Key Observations:**
  - **Engine Temp (cTemp):** Mean **130.6°C**, but upper capped at **255°C**, indicating sensor overrange or encoding
  - **RPM:** Mean **994.07**, peak **15,391**
  - **Speed:** Mean **48.5 km/h**, but wide spread with outliers up to **448.6 km/h**
  - **MAF & tPos:** Vary significantly, suggest inconsistent units
  - **accData:** Very high cardinality (5.5M unique strings) → Likely byte-level accelerometer logs

- **Notable Sample Entry:**
  ```plaintext
  Time: 2017-10-09 16:06:36 | RPM: 1200 | Speed: 36 km/h | Load: 29.02 | Source: LEVIN combined_1
  ```

---

##### ✅ `combined_2.csv`

- **Shape:** 1,322,388 rows × 18 columns  
- **Columns:** Similar to `combined_1.csv` but without `mx`, `my`, `mz`

- **Missing Values:**
  - ~929,000 values missing in major sensor columns (~70%)
  - `accData` also missing ~200K values

- **Key Observations:**
  - Many columns (e.g., `battery`, `cTemp`, `rpm`) have static value **255**, indicating sensor encoding saturation
  - **Speed:** Mean **75.96 km/h**, higher than `combined_1.csv`
  - **RPM, MAF, ELoad, tPos:** Flatlined at **255**, suggesting placeholder or faulty sensor session

- **Notable Sample Entry:**
  ```plaintext
  Time: 2017-08-30 08:08:15 | RPM: 255 | Speed: 255 km/h | Throttle Pos: 255 | Source: LEVIN combined_2
  ```

---

##### 📌 Key Takeaways

- **Rich time-series data** with trip-based and real-time vehicle diagnostics.
- Many sensors exhibit capped values (255), suggesting encoding/sensor limits.
- **accData** and motion vectors (`mx`, `my`, `mz`) can be useful for anomaly or driver behavior detection.
- `combined_1.csv` contains more reliable and usable sensor data than `combined_2.csv`.
- Missing values and over-capped values must be handled before training any model.
- Dataset is ideal for multi-modal analysis: combining vehicle health, sensor streams, and driving events.